In [1]:
# importing libraries
import os 
import time
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
import ProbForecastFollmerProcess as pffp

from tqdm.notebook import tqdm
from torch.distributions import MultivariateNormal

In [ ]:
# setting plotting style and defining the device
plt.style.use('ggplot')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_folder = "./data_store/multi_modal_jump_diffusion"
print('Computing on ' + str(device))

In [ ]:
# setting reproducibility
reproducible = True
SEED = 1024 if reproducible else int(time.time())
pffp.utils.ensure_reproducibility(SEED)

In [ ]:
# defining number of dimensions
dim = 2

# defining base gaussian component
mean_0 = torch.tensor([5.0, 0.0], device = device)
cov_0 = torch.tensor([[1.5, 0.0], [0.0, 0.1]], device = device)

# defining number of mixture models
K = 5

# defining rotation angle
theta = torch.tensor([2.0*np.pi/K], device = device)

# defining rotation matrix
R_theta = lambda k: torch.tensor([[torch.cos(k*theta), -torch.sin(k*theta)], [torch.sin(k*theta), torch.cos(k*theta)]], device = device)
rotations = [R_theta(k) for k in range(K)]

# defining mixture
gmm_params = [{"mean": torch.matmul(R, mean_0), "cov": torch.matmul(torch.matmul(R, cov_0), R.T)} for R in rotations]
gmm_components = [MultivariateNormal(params["mean"], params["cov"]) for params in gmm_params]
print(gmm_params)

In [ ]:
# settings of the simulation
delta_t = torch.tensor([1e-2], device = device)
observation_interval = 50

# defining number of observations and iterations
num_iters = int(5e6) 
num_observations = int(num_iters / observation_interval)

# rate of the poisson process
poisson_rate = torch.tensor([2.0], device = device)

# defining the configuration dictionary
simulation_conf = {
    "delta_t": delta_t,
    "observation_interval": observation_interval,
    "num_iters": num_iters,
    "num_observations": num_observations,
    "poisson_rate": poisson_rate,
    "gmm_components": gmm_components,
    "R_theta": R_theta,
    "K": K,
    "dim": dim,
    "device": device
}

# checking if the simulation data is stored 
if len(os.listdir(data_folder)) == 0:
    print("Data not found, running the simulation")
    # simulating the trajectory we will use for training
    train_state_store, train_observation_store = pffp.utils.simulate_jump_diffusion(simulation_conf)
    # simulating the trajectory we will use for testing
    test_state_store, test_observation_store = pffp.utils.simulate_jump_diffusion(simulation_conf)
    # saving the simulations so that we won't have to run them again
    torch.save(train_state_store, os.path.join(data_folder, "train_states.pt"))
    torch.save(train_observation_store, os.path.join(data_folder, "train_observations.pt"))
    torch.save(test_state_store, os.path.join(data_folder, "test_states.pt"))
    torch.save(test_observation_store, os.path.join(data_folder, "test_observations.pt"))
else:
    print("Loading simulation data from disk")
    # loading training states and observations
    train_state_store = torch.load(os.path.join(data_folder, "train_states.pt"))
    train_observation_store = torch.load(os.path.join(data_folder, "train_observations.pt"))
    # loading testing states and observations
    test_state_store = torch.load(os.path.join(data_folder, "test_states.pt"))
    test_observation_store = torch.load(os.path.join(data_folder, "test_observations.pt"))
# printing shape of the data
print(f"{train_state_store.shape=}, {train_observation_store.shape=}")
print(f"{test_state_store.shape=}, {test_observation_store.shape=}")

In [ ]:
# defining figure and axes
fig, axes = plt.subplots(1, 2)

# plotting the states of the simulated dynamics
x = train_state_store[:,0].detach().cpu()
y = train_state_store[:,1].detach().cpu()
pffp.utils.plot_density(x, y, fig, axes[0], "States")
# plotting the observations of the simulated dynamics
x = train_observation_store[:,0].detach().cpu()
y = train_observation_store[:,1].detach().cpu()
pffp.utils.plot_density(x, y, fig, axes[1], "Observations")

In [ ]:
# defining lag
tau_mul = 1 # 3 4
# sampling function for training data
get_train_data = lambda N: (i.to(device) for i in pffp.utils.sample_observations(N, train_observation_store, tau_mul))
# sampling function for testing data
get_test_data = lambda N: (i.to(device) for i in pffp.utils.sample_observations(N, test_observation_store, tau_mul))

In [ ]:
# defining data configurations
data = {    
    "train": get_train_data, 
    "test": get_train_data, 
}

# defining sampling configurations
sample = {
    "g": pffp.interpolant["sigma"],#pffp.utils.g_follmer, 
    "N": 1000
}

# learning standardization means and standard deviations
standardize = True
standardization = {
    'state_mean': torch.mean(train_observation_store, 0).to(device) if standardize else torch.zeros(dim).to(device),
    'state_std': torch.std(train_observation_store, 0).to(device) if standardize else torch.ones(dim).to(device) ,
}
print(standardization)

# defining network configurations
net_config = {
    "layers": [500]*5, 
    "standardization": standardization
}

# defining state configurations 
state = {
    "dim": dim
}

# defining optimization configurations
optim_config = {
    'minibatch': 1, 
    'num_obs_per_batch': 1000, 
    'num_iterations': 300,
    'learning_rate' : 0.001,
    'num_mc_samples': 100 
}

# defining model
model = pffp.core.model(data, sample, state, pffp.utils.interpolant, pffp.utils.velocity, net_config, device = "cuda")

# printing model
print(model)
# training model
model.train(optim_config)

In [ ]:
# retrieving loss and learning rates
losses = model.loss.detach().cpu()
lrs = model.lrs.detach().cpu()
# defining axes and figure
fig, axes = plt.subplots(1, 2)
# plotting loss
axes[0].set_xlabel("Iteration")
axes[0].set_ylabel("Loss")
axes[0].plot(losses)
# plotting learning rates
axes[1].set_xlabel("Iteration")
axes[1].set_ylabel("Learning rate")
axes[1].plot(lrs)

In [ ]:
# sampling configuration
sample_config = {
    "minibatch": 1, 
    "num_obs_per_batch": 1000,
    "num_samples_per_obs": 1
}

# running sampling
(X0, X1), samples = model.sample(sample_config)
print(f"{X0.shape=}, {X1.shape=}, {samples.shape=}")

In [ ]:
%matplotlib inline
# retrieving the data
x_hat, y_hat = samples[0, :, 0].detach().cpu(), samples[0, :, 1].detach().cpu()
x0, y0 = X0[:, 0].detach().cpu(), X0[:, 1].detach().cpu()
x1, y1 = X1[:, 0].detach().cpu(), X1[:, 1].detach().cpu()

# defining axes and figure
fig, axes = plt.subplots(1, 3)

# plotting the distributions
pffp.utils.plot_density(x0, y0, fig, axes[0], title = "initial states", bins = 50)
pffp.utils.plot_density(x1, y1, fig, axes[1], title = "next states", bins = 50)
pffp.utils.plot_density(x_hat, y_hat, fig, axes[2], title = "sampled states", bins = 50)
fig.show()

In [ ]:
# converting to angular coordinates
theta_0 = pffp.utils.vec2angle(x0,y0)
theta_1 = pffp.utils.vec2angle(x1,y1)
theta_hat = pffp.utils.vec2angle(x_hat,y_hat)

# defining figure and axes
fig, axes = plt.subplots(1, 3, figsize = (10, 5))

# plotting the angular distributions
hist0 = axes[0].hist(theta_0, density = True, bins = 200)
hist1 = axes[1].hist(theta_1, density = True, bins = 200)
hist_hat = axes[2].hist(theta_hat, density = True, bins = 200)

# showing the figure
fig.show()

In [ ]:
# autoregressive sampling configuration
sample_config = {
    "minibatch": 1, 
    "num_obs_per_batch": 1,
    "num_samples_per_obs": 1,
    "ar_steps": 100
}

# running autoregressive sampling
X0, ar_samples = model.sample_autoregressive(sample_config)
print(f"{X0.shape=}, {ar_samples.shape=}")

In [ ]:
%matplotlib inline
# retrieving the data
x_hat, y_hat = ar_samples[:, 0, 0].detach().cpu(), ar_samples[:, 0, 1].detach().cpu()

# defining axes and figure
fig, axes = plt.subplots()

# plotting the distributions
pffp.utils.plot_density(x_hat, y_hat, fig, axes, title = "sampled states", bins = 50)
axes.scatter(X0[0, 0].detach().cpu(), X0[0, 1].detach().cpu(), color = "red")
fig.show()